In [8]:
import pandas as pd
import re
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta


from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from catboost import CatBoostClassifier, CatBoostRegressor
from xgboost import XGBClassifier, XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, roc_auc_score
from sklearn.metrics import r2_score, root_mean_squared_error, mean_squared_error, mean_absolute_percentage_error, mean_absolute_error

## 데이터

In [5]:
ads_segment = pd.read_csv('/Users/t2023-m0052/Documents/GitHub/final_project/data/ads_segment.csv')
ads_segment = ads_segment.iloc[:, 1:]
ads_segment

,ads_idx,media_count,user_count,total_clicks,total_conversions,ads_category,domain,ads_os_type,ads_order,ctit_mean,...,performance_level,ads_level,media_score,conv_score,clicks_score,stability_score,cvr_score,total_score,ads_size,cluster
0,6508,1,1,2,0,10,금융,7,2381500,NaN,...,LOW,SMALL_LOW,1,0,0,0,0,1,SMALL,0
1,6985,2,30,46,0,7,금융,7,1201938,NaN,...,LOW,MEDIUM_LOW,2,0,0,3,0,5,MEDIUM,0
2,8327,1,6,11,0,10,미디어/컨텐츠,7,1673458,NaN,...,LOW,SMALL_LOW,1,0,0,3,0,4,SMALL,0
3,9264,1,1,1,0,10,금융,7,2372800,NaN,...,LOW,SMALL_LOW,1,0,0,0,0,1,SMALL,0
4,9716,2,4,6,1,8,생활,2,2699900,74.000000,...,LOW,MEDIUM_LOW,2,0,1,1,0,4,SMALL,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9512,446899,2,52,63,49,4,커머스,7,1369916,42.102041,...,HIGH,LARGE_HIGH,2,3,2,0,2,9,LARGE,0
9513,446900,2,44,59,41,4,식당/카페,7,1369915,48.512195,...,HIGH,LARGE_HIGH,2,3,2,0,1,8,LARGE,0
9514,446902,1,2,2,0,8,미디어/컨텐츠,7,2735900,NaN,...,LOW,SMALL_LOW,1,0,0,0,0,1,SMALL,0
9515,446908,1,1,1,0,0,금융,2,1682806,NaN,...,LOW,SMALL_LOW,1,0,0,0,0,1,SMALL,0


In [6]:
media_portfolio = pd.read_csv('/Users/t2023-m0052/Documents/GitHub/final_project/data/media_portfolio.csv')
media_portfolio = media_portfolio.iloc[:, 1:]
media_portfolio

,mda_idx,user_count,total_clicks,total_conversions,first_click,last_click,days_active,daily_avg_conversions,LARGE,MEDIUM,...,domain_비영리/공공_pct,domain_생활_pct,domain_식당/카페_pct,domain_식음료_pct,domain_운동/스포츠_pct,domain_운세_pct,domain_의료/건강_pct,domain_채용_pct,domain_커머스_pct,expected_total_profit
0,12,75700,232438,82938,2025-07-26 00:00:02,2025-08-25 11:22:16,31,2675.419355,7,0,...,16.666667,12.500000,0.00000,16.666667,0.000000,0.000000,4.166667,0.000000,12.500000,3003668
1,14,31834,84763,22983,2025-07-26 00:01:22,2025-08-25 11:22:40,31,741.387097,31,5,...,0.000000,13.559322,0.00000,3.389831,0.000000,0.000000,3.389831,1.694915,5.084746,2733547
2,18,2825,5961,250,2025-07-26 00:00:49,2025-08-25 11:13:19,31,8.064516,19,0,...,0.000000,8.695652,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,4.347826,182366
3,22,37668,83355,35232,2025-07-26 00:00:31,2025-08-25 11:14:50,31,1136.516129,67,58,...,3.012048,11.445783,0.60241,4.216867,1.204819,7.831325,4.819277,1.204819,7.228916,2541770
4,26,402,678,70,2025-07-26 01:45:31,2025-08-25 10:22:36,31,2.258065,10,5,...,0.000000,10.526316,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,10.526316,2690
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
184,1050,1,1,1,2025-08-11 21:03:48,2025-08-11 21:03:48,1,1.000000,1,0,...,0.000000,0.000000,0.00000,0.000000,0.000000,100.000000,0.000000,0.000000,0.000000,5
185,1051,1,1,0,2025-08-22 11:49:35,2025-08-22 11:49:35,1,0.000000,0,0,...,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0
186,1055,1,3,0,2025-07-27 22:52:51,2025-07-27 22:52:57,1,0.000000,0,0,...,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0
187,1059,1,6,0,2025-08-08 14:34:10,2025-08-08 14:36:18,1,0.000000,0,1,...,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0


In [18]:
part_earn = pd.read_csv('/Users/t2023-m0052/Documents/GitHub/final_project/data/참여적립조인.csv')
part_earn = part_earn.iloc[:, 1:]
part_earn

,click_key,ads_idx,dvc_idx,mda_idx,pub_sub_rel_id,contract_price,media_price,click_day,click_time,click_date,...,network,user_ip,rwd_idx,ads_code,advid,adv_cost,earn_cost,regdate,ctit,conversion
0,000000d54b9faad47ee99d6cd3cf53894dd4baa5,313780,61906528,539,144350110,6000,4500,2025-08-17,21,2025-08-17 21:07:37,...,0,16.184.28.219,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,000002b4d92f7648b455877c2676452efcd22a09,412426,34422806,58,46032732,180,170,2025-07-26,2,2025-07-26 02:18:24,...,NaN,35.78.117.76,238313226.0,yuRe4YoPuZ,40ceb01c-9b60-4452-bacd-0e061fb18ab4,180.0,170.0,2025-07-26 02:19:21,57.0,1
2,0000057e97361ff3d0263aaecee34cfaa3ba30fb,443660,38366075,808,1,170,120,2025-08-12,18,2025-08-12 18:17:59,...,WIFI,39.7.55.192,239017177.0,WuIp8HKmt9,73793f44-a471-4351-ae1c-d09accaf223a,170.0,120.0,2025-08-12 18:19:21,82.0,1
3,00000607f60139015da3ee1dd5499db3faa100dc,360192,61894110,539,144350110,6000,4500,2025-08-17,3,2025-08-17 03:35:02,...,0,211.252.100.157,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4,0000066bc25d4a6d147c27326cf972a4de88024e,372307,61956954,539,144350110,15600,11700,2025-08-18,8,2025-08-18 08:01:26,...,0,3.38.148.211,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16854860,fffff9f3a33b1596ae3216eaee20a9fe383b4778,264409,37945741,539,144350110,5400,3800,2025-08-16,23,2025-08-16 23:57:32,...,0,122.32.24.115,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
16854861,fffffb2a049908716493ba68a0754633949a9e02,369816,57367216,539,144350110,3600,2700,2025-08-18,15,2025-08-18 15:16:57,...,0,13.125.212.103,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
16854862,fffffc10523f96c3a470ef1c9ad6b930cb5d27b5,444350,61242190,58,33306660,180,170,2025-08-20,1,2025-08-20 01:02:48,...,NaN,43.206.110.22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
16854863,fffffcdccdba2775b47c2f01b9659decad98419f,360192,61903879,539,144350110,6000,4500,2025-08-15,12,2025-08-15 12:04:56,...,0,115.40.102.142,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [16]:
domain = pd.read_csv('/Users/t2023-m0052/Documents/GitHub/final_project/data/광고도메인리스트.csv')
domain = domain.drop(['Unnamed: 0','ads_icon_img', 'ads_summary', 'ads_type'# , #'ads_sdate', 
                         #'ads_age_min', 'ads_age_max'
                         ], axis = 1)
domain

,ads_idx,ads_code,aff_idx,adv_idx,ads_category,ads_name,ads_save_way,ads_day_cap,ads_sdate,ads_edate,ads_age_min,ads_age_max,ads_os_type,ads_contract_price,ads_reward_price,ads_order,ads_rejoin_type,regdate,domain
0,6508,3u4lOKJAek,13,335,10,스마트피싱보호,서비스 가입,False,2020-04-23 17:00:00,9999-12-31 00:00:00,0,100,7,1400,1100,2381500,NONE,2020-04-23 17:53:01 UTC,금융
1,6985,SijDrtH4GV,13,335,7,DB손해보험 다이렉트,내차 보험료 조회,False,2020-06-30 15:00:00,9999-06-06 00:00:00,0,100,7,2700,2300,1201938,NONE,2020-06-30 15:22:59 UTC,금융
2,8327,k1RZftVHoW,13,335,10,파일썬,첫 결제,False,2020-11-10 11:21:00,9999-12-31 23:59:00,0,100,7,3200,2420,1673458,NONE,2020-11-10 11:21:43 UTC,미디어/컨텐츠
3,9264,EGZcaVOtlc,23,220,10,하나 가득담은 운전자보험가입,보험 가입 완료,False,2020-11-30 00:00:00,9999-12-31 23:59:59,0,100,7,11000,8250,2372800,NONE,2020-11-30 15:16:45 UTC,금융
4,9716,NwZZq45Kgb,6,10,8,베이비러브 참여 신청,참여신청 (25세 ~ 55세),False,2021-01-11 16:00:00,2029-11-29 00:00:00,25,55,2,600,480,2699900,NONE,2021-01-11 16:11:27 UTC,생활
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9512,446899,U80Aed1tRq,86,1655,4,원스토몰 180212,네이버스마트스토어 퀴즈,False,2025-08-25 09:37:27,9999-12-31 23:59:59,0,100,7,18,12,1369916,ADS_CODE_DAILY_UPDATE,2025-08-25 09:37:27 UTC,커머스
9513,446900,dfX4EjOsTE,86,1655,4,서빙카트 식당 업소용 주방 서빙카 다용도 서빙카트 소형 2단 기본고무바퀴 180213,네이버스마트스토어 퀴즈,False,2025-08-25 09:42:08,9999-12-31 23:59:59,0,100,7,18,12,1369915,ADS_CODE_DAILY_UPDATE,2025-08-25 09:42:08 UTC,식당/카페
9514,446902,rc7EZrAoQw,57,84,8,[네이버카페가입인사] 대장TV,참여,False,2025-08-25 10:00:00,2030-12-27 00:00:00,0,100,7,200,150,2735900,NONE,2025-08-25 10:06:15 UTC,미디어/컨텐츠
9515,446908,HRMcs0VIjL,13,335,0,NOL,최초 오픈,False,2025-08-25 10:37:11,9999-12-31 23:59:59,0,100,2,230,180,1682806,NONE,2025-08-25 10:37:10 UTC,금융


### 클로드 코드

In [12]:

class AdConversionBaseline:
    def __init__(self):
        self.models = {}
        self.scalers = {}
        self.label_encoders = {}
        self.media_stats = {}
        self.category_stats = {}
        
    def prepare_features(self, df):
        """특성 엔지니어링"""
        features = df.copy()
        
        # 1. 광고 기본 특성
        features['ad_duration'] = (pd.to_datetime(features['end_date']) - 
                                 pd.to_datetime(features['start_date'])).dt.days
        features['reward_amount_log'] = np.log1p(features['reward_amount'])
        features['start_weekday'] = pd.to_datetime(features['start_date']).dt.dayofweek
        features['start_month'] = pd.to_datetime(features['start_date']).dt.month
        
        # 2. 보상금액 구간화
        reward_bins = [0, 100, 500, 1000, 5000, float('inf')]
        features['reward_tier'] = pd.cut(features['reward_amount'], 
                                       bins=reward_bins, labels=[1,2,3,4,5])
        
        return features
    
    def calculate_historical_stats(self, conversion_data):
        """과거 성과 통계 계산"""
        # 매체별 통계
        media_stats = conversion_data.groupby('media').agg({
            'conversions': ['mean', 'std', 'count', 'sum'],
            'conversion_rate': 'mean',
            'avg_payout': 'mean'
        }).round(4)
        media_stats.columns = ['_'.join(col) for col in media_stats.columns]
        
        # 카테고리별 통계  
        category_stats = conversion_data.groupby('category').agg({
            'conversions': ['mean', 'std', 'count'],
            'conversion_rate': 'mean'
        }).round(4)
        category_stats.columns = ['_'.join(col) for col in category_stats.columns]
        
        # 매체 × 카테고리 교차 통계
        cross_stats = conversion_data.groupby(['media', 'category']).agg({
            'conversions': 'mean',
            'conversion_rate': 'mean'
        }).round(4)
        cross_stats.columns = ['cross_conversions_mean', 'cross_conversion_rate_mean']
        
        return media_stats, category_stats, cross_stats
    
    def create_training_data(self, ad_info, conversion_history):
        """학습용 데이터셋 생성"""
        training_data = []
        
        for _, ad in ad_info.iterrows():
            # 해당 광고의 모든 매체별 실적 가져오기
            ad_conversions = conversion_history[
                conversion_history['ad_code'] == ad['ad_code']
            ]
            
            # 모든 매체에 대해 레코드 생성 (전환이 없는 매체는 0)
            all_medias = conversion_history['media'].unique()
            
            for media in all_medias:
                media_conversion = ad_conversions[ad_conversions['media'] == media]
                
                record = {
                    'ad_code': ad['ad_code'],
                    'category': ad['category'],
                    'reward_amount': ad['reward_amount'],
                    'can_retry': ad['can_retry'],
                    'start_date': ad['start_date'],
                    'end_date': ad['end_date'],
                    'media': media,
                    'conversions': media_conversion['conversions'].sum() if len(media_conversion) > 0 else 0
                }
                training_data.append(record)
        
        return pd.DataFrame(training_data)
    
    def add_historical_features(self, df, media_stats, category_stats, cross_stats):
        """과거 성과 기반 특성 추가"""
        # 매체별 과거 성과
        df = df.merge(media_stats, left_on='media', right_index=True, how='left')
        
        # 카테고리별 과거 성과
        df = df.merge(category_stats, left_on='category', right_index=True, how='left')
        
        # 교차 특성
        df = df.merge(cross_stats, left_on=['media', 'category'], right_index=True, how='left')
        
        # 결측치 처리 (새로운 조합의 경우 전체 평균으로)
        numeric_cols = df.select_dtypes(include=[np.number]).columns
        for col in numeric_cols:
            if df[col].isna().any():
                df[col].fillna(df[col].mean(), inplace=True)
        
        return df
    
    def fit(self, ad_info, conversion_history):
        """모델 학습"""
        print("📊 과거 성과 통계 계산 중...")
        media_stats, category_stats, cross_stats = self.calculate_historical_stats(conversion_history)
        print("미디어 스탯\n",media_stats)
        print("카테고리 스탯\n",category_stats)
        print("크로스 스탯\n",cross_stats)

        print("🔧 학습 데이터 생성 중...")
        training_data = self.create_training_data(ad_info, conversion_history)
        training_data = self.prepare_features(training_data)
        training_data = self.add_historical_features(training_data, media_stats, category_stats, cross_stats)
        print("training data\n", training_data)

        # 저장해둠
        self.media_stats = media_stats
        self.category_stats = category_stats
        self.cross_stats = cross_stats
        
        print("🎯 특성 전처리 중...")
        # 범주형 변수 인코딩
        categorical_features = ['category', 'media', 'can_retry', 'reward_tier']
        for col in categorical_features:
            if col in training_data.columns:
                le = LabelEncoder()
                training_data[f'{col}_encoded'] = le.fit_transform(training_data[col].astype(str))
                self.label_encoders[col] = le
        
        # 수치형 특성 선택
        feature_cols = [
            'reward_amount', 'reward_amount_log', 'ad_duration', 
            'start_weekday', 'start_month'
        ]
        # 인코딩된 범주형 변수들 추가
        feature_cols.extend([f'{col}_encoded' for col in categorical_features if col in training_data.columns])
        # 과거 성과 특성들 추가
        stat_cols = [col for col in training_data.columns if any(x in col for x in ['_mean', '_std', '_count', '_sum', 'cross_'])]
        feature_cols.extend(stat_cols)
        
        X = training_data[feature_cols].fillna(0)
        y = training_data['conversions']
        
        print(f"📈 특성 개수: {len(feature_cols)} \n{feature_cols}")
        print(f"📊 학습 샘플 수: {len(X)}")
        
        # 스케일링
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X)
        self.scalers['features'] = scaler
        self.feature_cols = feature_cols
        
        # 데이터 분할
        X_train, X_test, y_train, y_test = train_test_split(
            X_scaled, y, test_size=0.2, random_state=42
        )
        
        # 모델 학습
        print("🤖 모델 학습 중...")
        
        # 1. 선형 회귀
        lr = LinearRegression()
        lr.fit(X_train, y_train)
        self.models['linear'] = lr
        
        # 2. 랜덤 포레스트
        rf = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
        rf.fit(X_train, y_train)
        self.models['random_forest'] = rf
        
        # 모델 평가
        print("\n📊 모델 성능 평가:")
        for name, model in self.models.items():
            y_pred = model.predict(X_test)
            mae = mean_absolute_error(y_test, y_pred)
            rmse = root_mean_squared_error(y_test, y_pred)
            r2 = r2_score(y_test, y_pred)
            print(f"{name:15} | MAE: {mae:.2f} | RMSE: {rmse:.2f} | R^2: {r2:.2f}")
        
        return self
    
    def predict_all_medias(self, new_ad, model_name='random_forest'):
        """새로운 광고에 대해 모든 매체의 전환 수 예측"""
        model = self.models[model_name]
        scaler = self.scalers['features']
        
        # 모든 매체에 대해 예측
        all_medias = self.media_stats.index.tolist()
        predictions = []
        
        for media in all_medias:
            # 예측용 데이터 생성
            pred_data = new_ad.copy()
            pred_data['media'] = media
            
            # 특성 엔지니어링
            pred_data = self.prepare_features(pd.DataFrame([pred_data]))
            pred_data = self.add_historical_features(
                pred_data, self.media_stats, self.category_stats, self.cross_stats
            )
            
            # 범주형 변수 인코딩
            for col, le in self.label_encoders.items():
                if col in pred_data.columns:
                    try:
                        pred_data[f'{col}_encoded'] = le.transform(pred_data[col].astype(str))
                    except ValueError:
                        # 새로운 카테고리의 경우 가장 빈번한 값으로 대체
                        pred_data[f'{col}_encoded'] = 0
            
            # 특성 추출 및 스케일링
            X = pred_data[self.feature_cols].fillna(0)
            X_scaled = scaler.transform(X)
            
            # 예측
            pred_conversions = model.predict(X_scaled)[0]
            pred_conversions = max(0, pred_conversions)  # 음수 방지
            
            predictions.append({
                'media': media,
                'predicted_conversions': round(pred_conversions, 2)
            })
        
        return pd.DataFrame(predictions)
    
    def recommend_top_medias(self, new_ad, top_k=10, model_name='random_forest'):
        """상위 K개 매체 추천"""
        predictions = self.predict_all_medias(new_ad, model_name)
        
        # 상위 K개 선택
        top_medias = predictions.nlargest(top_k, 'predicted_conversions')
        
        print(f"🎯 '{new_ad['category']}' 카테고리 광고 (보상: {new_ad['reward_amount']}원)")
        print("=" * 60)
        print(f"{'순위':<4} {'매체명':<20} {'예상 전환 수':<12} {'신뢰도'}")
        print("-" * 60)
        
        for idx, (_, row) in enumerate(top_medias.iterrows(), 1):
            media = row['media']
            pred = row['predicted_conversions']
            
            # 신뢰도 계산 (해당 매체의 과거 데이터 양 기반)
            historical_count = self.media_stats.loc[media, 'conversions_count'] if media in self.media_stats.index else 0
            confidence = min(100, historical_count * 2)  # 간단한 신뢰도 지표
            
            print(f"{idx:<4} {media:<20} {pred:<12} {confidence:.0f}%")
        
        return top_medias

# 사용 예시 및 테스트
def create_sample_data():
    """샘플 데이터 생성"""
    
    # 매체 리스트
    medias = ['Google', 'Facebook', 'Instagram', 'YouTube', 'TikTok', 
              'KakaoTalk', 'Naver', 'Coupang', 'Baemin', 'Toss']
    
    # 카테고리 리스트  
    categories = ['Shopping', 'Game', 'Finance', 'Food', 'Travel', 'Health']
    
    # 광고 정보 샘플
    ad_info = pd.DataFrame({
        'ad_code': [f'AD_{i:03d}' for i in range(1, 101)],
        'category': np.random.choice(categories, 100),
        'reward_amount': np.random.choice([100, 300, 500, 1000, 2000, 5000], 100),
        'can_retry': np.random.choice([True, False], 100),
        'start_date': pd.date_range('2024-01-01', periods=100, freq='3D'),
        'end_date': pd.date_range('2024-01-08', periods=100, freq='3D')
    })
    
    # 과거 전환 이력 샘플 (실제로는 복잡한 집계 테이블)
    conversion_history = []
    for _, ad in ad_info.iterrows():
        for media in medias:
            # 매체별로 다른 성과 특성 부여
            base_conversion = np.random.poisson(5) if media in ['Google', 'Facebook'] else np.random.poisson(2)
            
            # 카테고리별 가중치
            category_weight = 1.5 if ad['category'] in ['Shopping', 'Game'] else 1.0
            
            # 보상금액별 가중치
            reward_weight = min(2.0, ad['reward_amount'] / 1000)
            
            conversions = int(base_conversion * category_weight * reward_weight)
            
            conversion_history.append({
                'ad_code': ad['ad_code'],
                'media': media,
                'category': ad['category'],
                'conversions': conversions,
                'conversion_rate': min(0.15, conversions / 100),
                'avg_payout': ad['reward_amount'] * 0.8
            })
    
    return ad_info, pd.DataFrame(conversion_history)

# 실행 예시
if __name__ == "__main__":
    # 샘플 데이터 생성
    print("🔧 샘플 데이터 생성 중...")
    ad_info, conversion_history = create_sample_data()
    print(ad_info)
    print(conversion_history)
    
    # 모델 학습
    model = AdConversionBaseline()
    model.fit(ad_info, conversion_history)
    
    # 새로운 광고에 대한 예측
    new_ad = {
        'category': 'Shopping',
        'reward_amount': 1000,
        'can_retry': True,
        'start_date': '2024-12-01',
        'end_date': '2024-12-31'
    }
    
    print("\n" + "="*80)
    print("🚀 새로운 광고에 대한 매체 추천")
    print("="*80)
    
    # 상위 10개 매체 추천
    recommendations = model.recommend_top_medias(new_ad, top_k=10)
    print(recommendations)

🔧 샘플 데이터 생성 중...
   ad_code category  reward_amount  can_retry start_date   end_date
0   AD_001   Health            100       True 2024-01-01 2024-01-08
1   AD_002     Food           2000       True 2024-01-04 2024-01-11
2   AD_003   Travel           5000       True 2024-01-07 2024-01-14
3   AD_004   Travel            300      False 2024-01-10 2024-01-17
4   AD_005     Food           1000      False 2024-01-13 2024-01-20
..     ...      ...            ...        ...        ...        ...
95  AD_096   Travel           5000      False 2024-10-12 2024-10-19
96  AD_097   Travel           2000       True 2024-10-15 2024-10-22
97  AD_098     Game           5000       True 2024-10-18 2024-10-25
98  AD_099   Travel           5000       True 2024-10-21 2024-10-28
99  AD_100     Food           1000      False 2024-10-24 2024-10-31

[100 rows x 6 columns]
    ad_code      media category  conversions  conversion_rate  avg_payout
0    AD_001     Google   Health            0             0.00        

In [19]:


def prepare_features(df):
    """특성 엔지니어링"""
    features = df.copy()
    
    # 1. 광고 기본 특성
    features['ad_duration'] = (pd.to_datetime(features['end_date']) - 
                             pd.to_datetime(features['start_date'])).dt.days
    features['reward_amount_log'] = np.log1p(features['reward_amount'])
    features['start_weekday'] = pd.to_datetime(features['start_date']).dt.dayofweek
    features['start_month'] = pd.to_datetime(features['start_date']).dt.month
    
    # 2. 보상금액 구간화
    reward_bins = [0, 100, 500, 1000, 5000, float('inf')]
    features['reward_tier'] = pd.cut(features['reward_amount'], 
                                   bins=reward_bins, labels=[1,2,3,4,5])
    
    return features

def calculate_historical_stats(conversion_data):
    """과거 성과 통계 계산"""
    # 매체별 통계
    media_stats = conversion_data.groupby('media').agg({
        'conversions': ['mean', 'std', 'count', 'sum'],
        'conversion_rate': 'mean',
        'avg_payout': 'mean'
    }).round(4)
    media_stats.columns = ['_'.join(col) for col in media_stats.columns]
    
    # 카테고리별 통계  
    category_stats = conversion_data.groupby('category').agg({
        'conversions': ['mean', 'std', 'count'],
        'conversion_rate': 'mean'
    }).round(4)
    category_stats.columns = ['_'.join(col) for col in category_stats.columns]
    
    # 매체 × 카테고리 교차 통계
    cross_stats = conversion_data.groupby(['media', 'category']).agg({
        'conversions': 'mean',
        'conversion_rate': 'mean'
    }).round(4)
    cross_stats.columns = ['cross_conversions_mean', 'cross_conversion_rate_mean']
    
    return media_stats, category_stats, cross_stats

def create_training_data(ad_info, conversion_history):
    """학습용 데이터셋 생성"""
    training_data = []
    
    for _, ad in ad_info.iterrows():
        # 해당 광고의 모든 매체별 실적 가져오기
        ad_conversions = conversion_history[
            conversion_history['ad_code'] == ad['ad_code']
        ]
        
        # 모든 매체에 대해 레코드 생성 (전환이 없는 매체는 0)
        all_medias = conversion_history['media'].unique()
        
        for media in all_medias:
            media_conversion = ad_conversions[ad_conversions['media'] == media]
            
            record = {
                'ad_code': ad['ad_code'],
                'category': ad['category'],
                'reward_amount': ad['reward_amount'],
                'can_retry': ad['can_retry'],
                'start_date': ad['start_date'],
                'end_date': ad['end_date'],
                'media': media,
                'conversions': media_conversion['conversions'].sum() if len(media_conversion) > 0 else 0
            }
            training_data.append(record)
    
    return pd.DataFrame(training_data)

def add_historical_features(df, media_stats, category_stats, cross_stats):
    """과거 성과 기반 특성 추가"""
    # 매체별 과거 성과
    df = df.merge(media_stats, left_on='media', right_index=True, how='left')
    
    # 카테고리별 과거 성과
    df = df.merge(category_stats, left_on='category', right_index=True, how='left')
    
    # 교차 특성
    df = df.merge(cross_stats, left_on=['media', 'category'], right_index=True, how='left')
    
    # 결측치 처리 (새로운 조합의 경우 전체 평균으로)
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    for col in numeric_cols:
        if df[col].isna().any():
            df[col].fillna(df[col].mean(), inplace=True)
    
    return df

def train_models(ad_info, conversion_history):
    """모델 학습"""
    print("📊 과거 성과 통계 계산 중...")
    media_stats, category_stats, cross_stats = calculate_historical_stats(conversion_history)
    
    print("🔧 학습 데이터 생성 중...")
    training_data = create_training_data(ad_info, conversion_history)
    training_data = prepare_features(training_data)
    training_data = add_historical_features(training_data, media_stats, category_stats, cross_stats)
    
    print("🎯 특성 전처리 중...")
    # 범주형 변수 인코딩
    label_encoders = {}
    categorical_features = ['category', 'media', 'can_retry', 'reward_tier']
    for col in categorical_features:
        if col in training_data.columns:
            le = LabelEncoder()
            training_data[f'{col}_encoded'] = le.fit_transform(training_data[col].astype(str))
            label_encoders[col] = le
    
    # 수치형 특성 선택
    feature_cols = [
        'reward_amount', 'reward_amount_log', 'ad_duration', 
        'start_weekday', 'start_month'
    ]
    # 인코딩된 범주형 변수들 추가
    feature_cols.extend([f'{col}_encoded' for col in categorical_features if col in training_data.columns])
    # 과거 성과 특성들 추가
    stat_cols = [col for col in training_data.columns if any(x in col for x in ['_mean', '_std', '_count', '_sum', 'cross_'])]
    feature_cols.extend(stat_cols)
    
    X = training_data[feature_cols].fillna(0)
    y = training_data['conversions']
    
    print(f"📈 특성 개수: {len(feature_cols)}")
    print(f"📊 학습 샘플 수: {len(X)}")
    
    # 스케일링
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    # 데이터 분할
    X_train, X_test, y_train, y_test = train_test_split(
        X_scaled, y, test_size=0.2, random_state=42
    )
    
    # 모델 학습
    print("🤖 모델 학습 중...")
    
    # 1. 선형 회귀
    lr = LinearRegression()
    lr.fit(X_train, y_train)
    
    # 2. 랜덤 포레스트
    rf = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
    rf.fit(X_train, y_train)
    
    models = {'linear': lr, 'random_forest': rf}
    
    # 모델 평가
    print("\n📊 모델 성능 평가:")
    for name, model in models.items():
        y_pred = model.predict(X_test)
        mae = mean_absolute_error(y_test, y_pred)
        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        print(f"{name:15} | MAE: {mae:.2f} | RMSE: {rmse:.2f}")
    
    # 모든 필요한 정보를 담은 딕셔너리 반환
    model_info = {
        'models': models,
        'scaler': scaler,
        'label_encoders': label_encoders,
        'feature_cols': feature_cols,
        'media_stats': media_stats,
        'category_stats': category_stats,
        'cross_stats': cross_stats
    }
    
    return model_info

def predict_all_medias(new_ad, model_info, model_name='random_forest'):
    """새로운 광고에 대해 모든 매체의 전환 수 예측"""
    model = model_info['models'][model_name]
    scaler = model_info['scaler']
    label_encoders = model_info['label_encoders']
    feature_cols = model_info['feature_cols']
    media_stats = model_info['media_stats']
    category_stats = model_info['category_stats']
    cross_stats = model_info['cross_stats']
    
    # 모든 매체에 대해 예측
    all_medias = media_stats.index.tolist()
    predictions = []
    
    for media in all_medias:
        # 예측용 데이터 생성
        pred_data = new_ad.copy()
        pred_data['media'] = media
        
        # 특성 엔지니어링
        pred_data = prepare_features(pd.DataFrame([pred_data]))
        pred_data = add_historical_features(
            pred_data, media_stats, category_stats, cross_stats
        )
        
        # 범주형 변수 인코딩
        for col, le in label_encoders.items():
            if col in pred_data.columns:
                try:
                    pred_data[f'{col}_encoded'] = le.transform(pred_data[col].astype(str))
                except ValueError:
                    # 새로운 카테고리의 경우 가장 빈번한 값으로 대체
                    pred_data[f'{col}_encoded'] = 0
        
        # 특성 추출 및 스케일링
        X = pred_data[feature_cols].fillna(0)
        X_scaled = scaler.transform(X)
        
        # 예측
        pred_conversions = model.predict(X_scaled)[0]
        pred_conversions = max(0, pred_conversions)  # 음수 방지
        
        predictions.append({
            'media': media,
            'predicted_conversions': round(pred_conversions, 2)
        })
    
    return pd.DataFrame(predictions)

def recommend_top_medias(new_ad, model_info, top_k=10, model_name='random_forest'):
    """상위 K개 매체 추천"""
    predictions = predict_all_medias(new_ad, model_info, model_name)
    media_stats = model_info['media_stats']
    
    # 상위 K개 선택
    top_medias = predictions.nlargest(top_k, 'predicted_conversions')
    
    print(f"🎯 '{new_ad['category']}' 카테고리 광고 (보상: {new_ad['reward_amount']}원)")
    print("=" * 60)
    print(f"{'순위':<4} {'매체명':<20} {'예상 전환 수':<12} {'신뢰도'}")
    print("-" * 60)
    
    for idx, (_, row) in enumerate(top_medias.iterrows(), 1):
        media = row['media']
        pred = row['predicted_conversions']
        
        # 신뢰도 계산 (해당 매체의 과거 데이터 양 기반)
        historical_count = media_stats.loc[media, 'conversions_count'] if media in media_stats.index else 0
        confidence = min(100, historical_count * 2)  # 간단한 신뢰도 지표
        
        print(f"{idx:<4} {media:<20} {pred:<12} {confidence:.0f}%")
    
    return top_medias

def create_sample_data():
    """샘플 데이터 생성"""
    
    # 매체 리스트
    medias = ['Google', 'Facebook', 'Instagram', 'YouTube', 'TikTok', 
              'KakaoTalk', 'Naver', 'Coupang', 'Baemin', 'Toss']
    
    # 카테고리 리스트  
    categories = ['Shopping', 'Game', 'Finance', 'Food', 'Travel', 'Health']
    
    # 광고 정보 샘플
    ad_info = pd.DataFrame({
        'ad_code': [f'AD_{i:03d}' for i in range(1, 101)],
        'category': np.random.choice(categories, 100),
        'reward_amount': np.random.choice([100, 300, 500, 1000, 2000, 5000], 100),
        'can_retry': np.random.choice([True, False], 100),
        'start_date': pd.date_range('2024-01-01', periods=100, freq='3D'),
        'end_date': pd.date_range('2024-01-08', periods=100, freq='3D')
    })
    
    # 과거 전환 이력 샘플 (실제로는 복잡한 집계 테이블)
    conversion_history = []
    for _, ad in ad_info.iterrows():
        for media in medias:
            # 매체별로 다른 성과 특성 부여
            base_conversion = np.random.poisson(5) if media in ['Google', 'Facebook'] else np.random.poisson(2)
            
            # 카테고리별 가중치
            category_weight = 1.5 if ad['category'] in ['Shopping', 'Game'] else 1.0
            
            # 보상금액별 가중치
            reward_weight = min(2.0, ad['reward_amount'] / 1000)
            
            conversions = int(base_conversion * category_weight * reward_weight)
            
            conversion_history.append({
                'ad_code': ad['ad_code'],
                'media': media,
                'category': ad['category'],
                'conversions': conversions,
                'conversion_rate': min(0.15, conversions / 100),
                'avg_payout': ad['reward_amount'] * 0.8
            })
    
    return ad_info, pd.DataFrame(conversion_history)

# 실행 예시
print("🔧 샘플 데이터 생성 중...")
ad_info, conversion_history = create_sample_data()

🔧 샘플 데이터 생성 중...


In [20]:
ad_info

,ad_code,category,reward_amount,can_retry,start_date,end_date
0,AD_001,Shopping,1000,True,2024-01-01,2024-01-08
1,AD_002,Food,5000,False,2024-01-04,2024-01-11
2,AD_003,Health,2000,False,2024-01-07,2024-01-14
3,AD_004,Food,2000,True,2024-01-10,2024-01-17
4,AD_005,Travel,2000,True,2024-01-13,2024-01-20
...,...,...,...,...,...,...
95,AD_096,Game,5000,True,2024-10-12,2024-10-19
96,AD_097,Food,2000,False,2024-10-15,2024-10-22
97,AD_098,Food,1000,False,2024-10-18,2024-10-25
98,AD_099,Finance,2000,True,2024-10-21,2024-10-28


In [21]:
conversion_history

,ad_code,media,category,conversions,conversion_rate,avg_payout
0,AD_001,Google,Shopping,13,0.13,800.0
1,AD_001,Facebook,Shopping,9,0.09,800.0
2,AD_001,Instagram,Shopping,4,0.04,800.0
3,AD_001,YouTube,Shopping,1,0.01,800.0
4,AD_001,TikTok,Shopping,3,0.03,800.0
...,...,...,...,...,...,...
995,AD_100,KakaoTalk,Shopping,0,0.00,400.0
996,AD_100,Naver,Shopping,0,0.00,400.0
997,AD_100,Coupang,Shopping,1,0.01,400.0
998,AD_100,Baemin,Shopping,2,0.02,400.0


In [23]:
part_earn.columns

Index(['click_key', 'ads_idx', 'dvc_idx', 'mda_idx', 'pub_sub_rel_id',
       'contract_price', 'media_price', 'click_day', 'click_time',
       'click_date', 'exp_day', 'network', 'user_ip', 'rwd_idx', 'ads_code',
       'advid', 'adv_cost', 'earn_cost', 'regdate', 'ctit', 'conversion'],
      dtype='object')

In [29]:
conversion_history2 = part_earn.groupby(['ads_idx', 'mda_idx']).agg(
    conversions = ('conversion', 'sum'),
    users = ('dvc_idx', 'nunique'),
    clicks = ('click_day', 'size'),
    contract_price = ('contract_price', 'first'),
    media_price = ('media_price', 'first'),
    mean_ctit = ('ctit', 'mean')
    ).reset_index()
conversion_history2

,ads_idx,mda_idx,conversions,users,clicks,contract_price,media_price,mean_ctit
0,6508,854,0,1,2,1400,1100,NaN
1,6985,22,0,22,36,2700,2300,NaN
2,6985,401,0,8,10,2700,2300,NaN
3,8327,761,0,6,11,3200,2420,NaN
4,9264,772,0,1,1,11000,8250,NaN
...,...,...,...,...,...,...,...,...
21793,446900,563,39,41,56,18,14,47.692308
21794,446902,270,0,2,2,200,105,NaN
21795,446908,22,0,1,1,230,180,NaN
21796,446909,562,0,4,4,18,14,NaN


In [ ]:
conversion_history2['conversion_rate'] = conversion_history2['conversions'] / conversion_history2['clicks']
conversion_history2.merge(domain, on = '')

In [ ]:
# 모델 학습
model_info = train_models(ad_info, conversion_history)

# 새로운 광고에 대한 예측
new_ad = {
    'category': 'Shopping',
    'reward_amount': 1000,
    'can_retry': True,
    'start_date': '2024-12-01',
    'end_date': '2024-12-31'
}

print("\n" + "="*80)
print("🚀 새로운 광고에 대한 매체 추천")
print("="*80)

# 상위 10개 매체 추천
recommendations = recommend_top_medias(new_ad, model_info, top_k=10)

# 모든 매체 예측 결과도 확인
print("\n📊 모든 매체 예측 결과:")
all_predictions = predict_all_medias(new_ad, model_info)
print(all_predictions.sort_values('predicted_conversions', ascending=False))